## 현재 Directory: /AIFactory_Infer/

## 최종 폴더 구성

In [ ]:
'''
├─2023_AI_Competition -> 동영상 데이터 폴더
├─datasets -> 학습 데이터 폴더
│  ├─additional_data
│  ├─full
│  ├─labels
│  ├─test
│  ├─train
│  │  ├─images
│  │  └─labels
│  └─valid
│      ├─images
│      └─labels
└─ultralytics
    ├─runs
    │  ├─base
    │  │  └─weights -> 기존 데이터로 학습한 가중치
    │  └─base2
    │      └─weights -> 추가데이터 + 기존데이터로 학습한 가중치
    ├─tests
    └─ultralytics
        ├─assets
        ├─cfg
        │  ├─.ipynb_checkpoints
        │  ├─datasets
        │  │  └─.ipynb_checkpoints
        │  ├─models
        │  │  ├─rt-detr
        │  │  │  └─.ipynb_checkpoints
        │  │  ├─v3
        │  │  ├─v5
        │  │  ├─v6
        │  │  └─v8
        │  ├─trackers
        │  │  └─.ipynb_checkpoints
        │  └─__pycache__
        ├─data
        ├─engine
        ├─hub
        ├─models
        │  ├─yolo
        │  │  ├─.ipynb_checkpoints
        │  │  ├─classify
        │  │  │  └─__pycache__
        │  │  ├─detect
        │  │  │  ├─.ipynb_checkpoints
        │  │  │  └─__pycache__
        │  │  ├─pose
        │  │  │  └─__pycache__
        │  │  ├─segment
        │  │  │  └─__pycache__
        │  │  └─__pycache__
        │  └─__pycache__
        ├─nn
        ├─trackers
        └─utils
'''

### 기본환경 설정 및 라이브러리 설치 <<SEED=42>>
### colab 환경에서 실행시 conda 명령어는 제외하고 실행하면 됨

In [ ]:
# !conda create -n AIFactory python=3.9.16
# !conda activate AIFactory
!pip3 install tqdm
!pip3 install opencv-python
!pip3 install ttach

### Original DATASET 다운로드 : AIFactory_Infer 안에 주최측에서 제공해준 데이터셋(dataset.zip, 2023_AI_Competition.egg)이 압축해제된 상태로 넣어둔 상황이라고 가정.

### datasets폴더 생성

In [ ]:
!mkdir datasets

### train test 폴더 모두 datasets 폴더 안으로 이동시키기

In [ ]:
%mv  './train' './datasets/'
%mv  './test' './datasets/'
%mv  './labels' './datasets/'

### train 폴더를 full 폴더로 이동

In [ ]:
%mv './datasets/train' './datasets/full'

### 여기까지 디렉토리 상황

In [ ]:
'''
├─2023_AI_Competition -> 동영상 데이터 폴더
├─datasets -> 학습 데이터 폴더
│  ├─full
│  ├─labels
│  ├─test
'''

### 데이터셋 Yolo 형식으로 구성 및 Train & Valid Split과 동영상 데이터 이미지 변환

In [ ]:
!python coco2yolo.py
!python train_valid_split.py
!python save_video.py

### 여기까지 디렉토리 상황

In [ ]:
'''
├─2023_AI_Competition -> 동영상 데이터 폴더
├─datasets -> 학습 데이터 폴더
│  ├─additional_data
│  ├─full
│  ├─labels
│  ├─test
│  ├─train
│  │  ├─images
│  │  └─labels
│  └─valid
│      ├─images
│      └─labels
'''

### Custom Yolo를 사용하기 위해 코드 다운로드

In [ ]:
%cd ultralytics_train
!git clone https://github.com/rigvedrs/YOLO-V8-CAM.git
!pip3 install -r requirements.txt

# 2. 학습을 진행하고 추론도 진행하고 싶은 경우
- [추가 데이터 없이 Train 진행] 실행
- inference_additional.py 파일 안에 수정 필요
- [학습한 모델로 추가 데이터 Inference 진행] 실행
- [추가데이터 Train에 추가하기] 실행
- [추가 데이터와 기존 데이터 Train 진행] 실행
- test_find_fish.py 파일 안에 수정 필요
- [Inference한 json파일 COCO 형태로 변환] 실행


train / val / inference 모두 cache = True 옵션을 사용하는데, cache = True 옵션은 이미지를 미리 ram에 올려놓는 것을 의미함\
따라서, 30GB의 충분한 디스크 용량이 있다면 cache = True를 적용해주고
디스크 용량이 충분하지 않다면, cache = False / Batch size 변경도 필요함

### 추가 데이터 없이 Train 진행

In [ ]:
!python train_find_fish_first.py

### 학습한 모델로 추가 데이터 Inference 진행

In [ ]:
!python inference_additional.py

### 추가데이터 Train에 추가하기

In [ ]:
%cd ..
!python move_additional.py

### 추가 데이터와 기존 데이터 Train 진행

In [ ]:
%cd ultralytics_train
!python train_find_fish_after.py

### 최종 Inference 진행

In [ ]:
!python test_find_fish.py

### Inference한 json파일 COCO 형태로 변환

In [ ]:
%cd ..
!python results_to_coco.py